In [ ]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [ ]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

In [ ]:
using TransformVariables
using TransformedLogDensities, LogDensityProblemsAD
t = as((μ = asℝ, σ = asℝ₊, τ = asℝ₊, θs = as(Array, 8)))
dimension(t)
lp = StanProblem(bs_model);
P = TransformedLogDensity(t, lp)
∇P = ADgradient(:ForwardDiff, lp);

D = 10;
initial_θ = rand(Uniform(-2, 2), D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.95, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

In [ ]:
constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
unc_sample_array = vcat([
    param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
]...);

In [ ]:
lol = hcat(samples...)'
conhmc = vcat([
    param_constrain(bs_model, collect(row))' for row in eachrow(lol)
]...);

In [ ]:
mean(unc_sample_array, dims=1)

In [ ]:
mean(conhmc, dims=1)